In [1]:
import pandas as pd
import numpy as np
import os
data_folder = '/Users/mkamruz/Public/SNL/Projects/5UTR/Data/'


In [2]:
df_combine = pd.read_csv(os.path.join(data_folder, 'combined_results.csv'))
print(df_combine.shape)
df_combine.head()

(10000000, 28)


,sequence,DNABERT2_cao2021,DNABERT2_hek,DNABERT2_hek_muscle,DNABERT2_hek_muscle_pc3,DNABERT2_hek_pc3,DNABERT2_mpra,DNABERT2_muscle,DNABERT2_muscle_pc3,DNABERT2_pc3,...,grover_pc3,utrlm_cao2021,utrlm_hek,utrlm_hek_muscle,utrlm_hek_muscle_pc3,utrlm_hek_pc3,utrlm_mpra,utrlm_muscle,utrlm_muscle_pc3,utrlm_pc3
0,GTTCTATAGACATGAATGATTTGCG,0,1,1,1,1,1,0,1,1,...,1,1.0,0,0,0,0,0,1,1,1
1,CATTGACTAGCCCCATCCAATTGTA,0,1,1,1,1,1,0,1,1,...,1,1.0,0,1,1,1,1,0,0,1
2,CAGGCCTGTGGACGGCAAGTCTAGT,1,1,1,1,1,1,0,1,1,...,1,0.0,1,1,1,1,1,1,1,1
3,AGTTCAGACGCTATCTCTCACGATT,1,1,1,1,0,1,0,1,1,...,1,0.0,1,1,1,1,0,0,1,1
4,CGAAAGGTTGCGATCTCAGTTGTGC,1,0,0,1,1,1,1,1,1,...,1,1.0,1,1,1,1,1,1,1,0


In [3]:
for col in df_combine.columns:
    if col == 'sequence': continue
    if col.startswith('DNABERT2'):
         print(f"Difference from {col} to DNABERT2_cao2021 is: {np.sum(np.absolute(df_combine[col] - df_combine['DNABERT2_cao2021']))}")
    elif col.startswith('grover'):
         print(f"Difference from {col} to grover_cao2021 is: {np.sum(np.absolute(df_combine[col] - df_combine['grover_cao2021']))}")
    elif col.startswith('utrlm'):
         print(f"Difference from {col} to utrlm_cao2021 is: {np.sum(np.absolute(df_combine[col] - df_combine['utrlm_cao2021']))}")

del [df_combine]

Difference from DNABERT2_cao2021 to DNABERT2_cao2021 is: 0
Difference from DNABERT2_hek to DNABERT2_cao2021 is: 3361445
Difference from DNABERT2_hek_muscle to DNABERT2_cao2021 is: 3656771
Difference from DNABERT2_hek_muscle_pc3 to DNABERT2_cao2021 is: 2820234
Difference from DNABERT2_hek_pc3 to DNABERT2_cao2021 is: 2860638
Difference from DNABERT2_mpra to DNABERT2_cao2021 is: 5617853
Difference from DNABERT2_muscle to DNABERT2_cao2021 is: 5426610
Difference from DNABERT2_muscle_pc3 to DNABERT2_cao2021 is: 2515903
Difference from DNABERT2_pc3 to DNABERT2_cao2021 is: 2494226
Difference from grover_cao2021 to grover_cao2021 is: 0.0
Difference from grover_hek to grover_cao2021 is: 4196735.0
Difference from grover_hek_muscle to grover_cao2021 is: 4023794.0
Difference from grover_hek_muscle_pc3 to grover_cao2021 is: 4116092.0
Difference from grover_hek_pc3 to grover_cao2021 is: 4326383.0
Difference from grover_mpra to grover_cao2021 is: 4930296.0
Difference from grover_muscle to grover_cao20

In [17]:
cao_folder = os.path.join(data_folder, 'cao2021')

def get_combined_dna(data_loc, comb_col='UTR'):

    df_train = pd.read_csv(os.path.join(data_loc, 'train.csv'))
    df_test = pd.read_csv(os.path.join(data_loc, 'test.csv'))
    df_dev = pd.read_csv(os.path.join(data_loc, 'dev.csv'))
    print(f"Train shape: {df_train.shape}, Test shape: {df_test.shape}, Dev shape: {df_dev.shape}")
    #display(df_train.head(2))
    #display(df_test.head())
    #display(df_dev.head())
    dna = set(df_train[comb_col]).union(set(df_test[comb_col])).union(set(df_dev[comb_col]))
    print(f"Total unique sequences in {data_loc.split('/')[-1]}: {len(dna)}")
    return dna

def compute_Jaccard(set1, set2):
    intersection = len(set1.intersection(set2))
    print(f"Intersection size: {intersection}")
    union = len(set1.union(set2))
    print(f"Union size: {union}")
    jaccard_index = intersection / union if union != 0 else 0
    return jaccard_index

In [21]:
dna_cao = get_combined_dna(cao_folder, 'UTR')
op = [f"Total unique sequences in cao_2021: {len(dna_cao)}"]
for f in ['hek',
 'muscle_pc3',
 'hek_pc3',
 'pc3',
 'hek_muscle',
 'hek_muscle_pc3',
 'muscle']:
    op.append(f"\nProcessing file: {f}")
    dna_f = get_combined_dna(os.path.join(cao_folder, 'ratios', f), 'UTR')
    op.append(f"Total unique sequences in pc3: {len(dna_f)}")
    t = compute_Jaccard(dna_cao, dna_f)
    op.append(f"Jaccard similarity between cao_2021 and {f}: {t:.4f}")
    op.append("-" * 50)

with open('jaccard_cao2021_vs_others.txt', 'w') as f:
    f.writelines('\n'.join(op))

Train shape: (39616, 2), Test shape: (39616, 3), Dev shape: (4953, 2)
Total unique sequences in cao2021: 28851
Train shape: (35549, 2), Test shape: (4444, 2), Dev shape: (4444, 2)
Total unique sequences in hek: 28151
Intersection size: 20631
Union size: 36371
Train shape: (35031, 2), Test shape: (3892, 2), Dev shape: (4865, 2)
Total unique sequences in muscle_pc3: 26455
Intersection size: 18632
Union size: 36674
Train shape: (64403, 2), Test shape: (7155, 2), Dev shape: (8945, 2)
Total unique sequences in hek_pc3: 30000
Intersection size: 21179
Union size: 37672
Train shape: (36009, 2), Test shape: (4502, 2), Dev shape: (4501, 2)
Total unique sequences in pc3: 28376
Intersection size: 19913
Union size: 37314
Train shape: (34617, 2), Test shape: (3846, 2), Dev shape: (4808, 2)
Total unique sequences in hek_muscle: 26206
Intersection size: 19262
Union size: 35795
Train shape: (67025, 2), Test shape: (7447, 2), Dev shape: (9309, 2)
Total unique sequences in hek_muscle_pc3: 30146
Intersect

In [14]:
cao_2021_vs_pc3_jaccard = compute_Jaccard(dna_cao, dna_pc3)
print(f"Jaccard similarity between cao_2021 and pc3: {cao_2021_vs_pc3_jaccard:.4f}")

Intersection size: 19913
Union size: 37314
Jaccard similarity between cao_2021 and pc3: 0.5337


In [15]:
os.listdir(os.path.join(cao_folder, 'ratios'))

['hek',
 'muscle_pc3',
 'hek_pc3',
 'pc3',
 'hek_muscle',
 'hek_muscle_pc3',
 'muscle']